# Model Formulation

This notebook can be used to run initial model with DCOPF and a basic budget variable. Before running this nodebook please refer to `Test-System-Data-Processing.ipynb` to preprocess generator, lines, buses, and loads data. 

In [15]:
using JuMP, Gurobi
using DataFrames, CSV

include(joinpath("capex_model", "expansion_tools.jl"))

save (generic function with 1 method)

In [53]:
function expansion(buses, lines, gens, loads, variability, P)
    """
    Solve capacity expansion with CATS.
    
    Inputs:
        buses -- dataframe with bus types and loads
        lines -- dataframe with transmission lines info
        gens -- dataframe with generator info and costs
        loads -- 
        variability --
        P --
    """   
    #### HELPER FUNCTIONS ####
    # Select a line column using i and j 
    line = (i, j, col) -> lines[lines_map[(i, j)], col]
    # Subset of all lines connected to bus i
    J = i -> lines.t_bus[lines.f_bus .== i]
    # Select all generators in G connected to bus i
    sel = (G, i) -> collect(G)[gens[collect(G),:bus] .== i]
    
    
    ##### SETS ####
    G = gens[:, :gen_id]                 # G: Set of all generators
    G_s = gens[gens.ess .== 1, :gen_id]     # G_s: Set of all energy storage system (ESS) generators

    
    N = 1:nrow(buses)            # N: Set of all network nodes
    L = collect(zip(             # L: Set of all lines/branches
        lines.f_bus, lines.t_bus))

    T = 1:ncol(loads)            # T: Set of all hours

    #### GLOBALS ####
    # Mapping from (i, j) tuple to line index l
    lines_map = Dict(zip(L, 1:length(L)))
    
    #### PARAMETERS ####
    θlim = π*(60/180)  # Absolute max angle limit in rad
    slack_bus = 416    # Set slack bus to one with many CTs
    
    storage_hrs = 4    # Storage Hours * Power Capacity = Energy capacity ∀ g ∈ G_s
    η_charge = 0.93    # Charging efficiency  

    ########################
    #####     MODEL    #####
    ########################
    
    CATS_Model = Model(() -> Gurobi.Optimizer())
    #set_optimizer_attribute(CATS_Model, "NumericFocus", 2)
    
    # Decision variables   
    @variables(CATS_Model, begin
        GEN[G,T] ≥ 0       # Generation of each generator 
        SHED[N,T] ≥ 0      # Load sheading at bus N
        THETA[N,T]         # Voltage phase angle of bus
        FLOW[L,T]          # Flows between all pairs of nodes
        SOC[G_s,T] ≥ 0     # ESS state of charge
        CHARGE[G_s,T] ≥ 0  # Chargeing of ESS 
    end)

    # Max and min generation constraints
    @constraint(CATS_Model, cMaxGenOld[g ∈ G, t ∈ T], 
        GEN[g,t] ≤ variability[g,t]*gens[g,:pmax])

    # Load shedding constraint
    @constraint(CATS_Model, cShed[i ∈ N, t ∈ T], 
        SHED[i,t] ≤ loads[i,t])
    
    #########################
    #### BATTERY STORAGE ####
    #########################
    # Charging must be less than max power capacity
    @constraint(CATS_Model, cMaxChargeOld[g ∈ G_s, t ∈ T], 
        CHARGE[g,t] ≤ gens[g,:pmax])
    
    # Energy capacity must be less ×4 the power capacity
    @constraint(CATS_Model, cMaxStateOfChargeOld[g ∈ G_s, t ∈ T], 
        SOC[g,t] ≤ storage_hrs*gens[g,:pmax])
    
    # SOC in the next time is a function of SOC in the pervious time
    # with circular wrapping for the first and last t ∈ P_i
    @constraint(CATS_Model, cStateOfCharge[g ∈ G_s, t ∈ T[2:24]],
        SOC[g,t] == SOC[g,t-1] + (CHARGE[g,t]*η_charge - GEN[g,t]/η_charge))
    
    ########################
    ######## DC OPF ########
    ########################
    # Create slack bus with theta=0
    fix.(THETA[slack_bus, T], 0)
    
    # Max line flow must be less than the nominal (RATE A) of the line
    @constraint(CATS_Model, cLineLimits[(i,j) ∈ L, t ∈ T], 
        FLOW[(i,j),t] ≤ line(i,j,:rate_a)) 
    
    # Angle limited to less than 60 degrees 
    @constraint(CATS_Model, cAngleLimitsMax[(i,j) ∈ L, t ∈ T], 
        (THETA[i,t] - THETA[j,t]) ≤  θlim)
    @constraint(CATS_Model, cAngleLimitsMin[(i,j) ∈ L, t ∈ T], 
        (THETA[i,t] - THETA[j,t]) ≥ -θlim)
                    
    # Flow is governed by angle difference between buses
    @constraint(CATS_Model, cLineFlows[(i,j) ∈ L, t ∈ T],
        FLOW[(i,j),t] == line(i,j,:sus)*(THETA[i,t] - THETA[j,t]));

    # Power must be balanced at all nodes
    # Σ(generation) + shedding - Σ(demand) - Σ charging = Σ(flows) ∀ n ∈ N
    @constraint(CATS_Model, cBalance[i ∈ N, t ∈ T], 
        sum(GEN[g,t] for g ∈ sel(G, i))
        + SHED[i,t] - loads[i,t] 
        - sum(CHARGE[g,t] for g ∈ sel(G_s, i))
         == sum(FLOW[(i,j),t] for j ∈ J(i)))

    #########################
    ####### OBJECTIVE #######
    #########################

    # The weighted operational costs of running each generator
    @expression(CATS_Model, eVariableCosts,
        sum(gens[g,:c1] * GEN[g,t] for g ∈ G, t ∈ T))
    
    # The weighted operational costs of shedding load/non-served energy (NSE)
    @expression(CATS_Model, eNSECosts,
        sum(1000 * SHED[i,t] for i ∈ N, t ∈ T))

    # Minimize the sum of investment and O&M costs 
	@expression(CATS_Model, eTotalCosts,
		eVariableCosts + eNSECosts)
    
	@objective(CATS_Model, Min, eTotalCosts)

    optimize!(CATS_Model)

    return CATS_Model
end

expansion (generic function with 1 method)

In [55]:
# 3b. Ramping

# TODOs
# - kmeans clustering
# - Fix weighting
# - Ramping (if time)
# - Save more model outputs

## EXTRA
# Quardatic costs



## Questions for Qi ##
# a) costs? How do you handel the fact that costs are quadratic? Also why do they include c0?
# b) 1e-8 * 100 * (2700000 / 20 + 12500) # What are the units of this? and why do batteries cost less than solar??


# 3 mins to solve 1 week of cap ex

In [57]:
#run_model(load, basic_capex, save, "san_diego_system", "test_senario")

In [59]:
(buses, lines, gens, loads, variability, P) = load("san_diego_system");

In [60]:
model = expansion(buses, lines, gens, loads, variability, P)

Set parameter Username
Set parameter LicenseID to value 2669913
Academic license - for non-commercial use only - expires 2026-05-22
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M2 Ultra
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 159399 rows, 68616 columns and 327780 nonzeros
Model fingerprint: 0x4cdf82b5
Coefficient statistics:
  Matrix range     [9e-01, 1e+07]
  Objective range  [7e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-03, 3e+03]
Presolve removed 139185 rows and 47861 columns
Presolve time: 0.11s
Presolved: 20214 rows, 24987 columns, 89651 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.01s

Barrier statistics:
 Free vars  : 865
 AA' NZ     : 1.929e+05
 Factor NZ  : 5.256e+05 (roughly 23 MB of memory)
 Factor Ops : 1.752e+07 (less than 1 second per iteration)
 Thread

A JuMP Model
├ solver: Gurobi
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 68616
├ num_constraints: 187215
│ ├ AffExpr in MOI.EqualTo{Float64}: 44343
│ ├ AffExpr in MOI.GreaterThan{Float64}: 29088
│ ├ AffExpr in MOI.LessThan{Float64}: 85968
│ ├ VariableRef in MOI.EqualTo{Float64}: 24
│ └ VariableRef in MOI.GreaterThan{Float64}: 27792
└ Names registered in the model
  └ :CHARGE, :FLOW, :GEN, :SHED, :SOC, :THETA, :cAngleLimitsMax, :cAngleLimitsMin, :cBalance, :cLineFlows, :cLineLimits, :cMaxChargeOld, :cMaxGenOld, :cMaxStateOfChargeOld, :cShed, :cStateOfCharge, :eNSECosts, :eTotalCosts, :eVariableCosts

In [63]:
save(model, "san_diego_system", "test_senario", buses, lines, gens, loads)

"/Users/adamsedlak/Documents/Julia/MAE243/final-project/models/san_diego_system/outputs/test_senario/costs.csv"